In [1]:
# # to create kernel
# !conda install -c anaconda ipykernel
# !python -m ipykernel install --user --name=llm

## Langchain Docs
[Quickstart](https://python.langchain.com/docs/get_started/quickstart)

In [3]:
import yaml
import os
print(os.getcwd())
with open(r"..\openai.yaml") as f:
    spec = yaml.safe_load(f)
    key = spec['openai']['key']

os.environ['OPENAI_API_KEY'] = key

c:\Users\afs95\repos\langchain_course


## LLMs

In [23]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9, openai_api_key=key)

In [24]:
llm.predict("What would be a good company name for a company that makes colorful socks?")
# >> Feetful of Fun

'\n\nColorful Creations Socks'

## Chat Models

In [17]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0, openai_api_key=key)
chat.predict_messages([HumanMessage(content="Translate this sentence from English to French. I love programming.")])
# >> AIMessage(content="J'aime programmer.", additional_kwargs={})

AIMessage(content="J'adore la programmation.", additional_kwargs={}, example=False)

In [18]:
chat.predict("Translate this sentence from English to French. I love programming.")
# >> J'aime programmer

"J'adore la programmation."

## Prompt Templates

### LLMs

In [19]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good name for a company that makes {product}?")
prompt.format(product="colorful socks")

'What is a good name for a company that makes colorful socks?'

### Chat Models

In [26]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

[SystemMessage(content='You are a helpful assistant that translates English to French.', additional_kwargs={}),
 HumanMessage(content='I love programming.', additional_kwargs={}, example=False)]

## Chains

### LLMs

In [28]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
chain.run("colorful socks")

'\n\nRainbow Socks Company'

### Chat Models

In [30]:
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0, openai_api_key=key)

template = "You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="French", text="I love programming.")

"J'adore la programmation."

## Agents

### LLMs

In [5]:
with open(r"..\openai.yaml") as f:
    spec = yaml.safe_load(f)
    serp_key = spec['serpapi']['key']

os.environ['SERPAPI_API_KEY'] = serp_key

In [33]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

# The language model we're going to use to control the agent.
llm = OpenAI(temperature=0, openai_api_key=key)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm, )

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Let's test it out!
agent.run("What was the high temperature in SF yesterday in Fahrenheit? What is that number raised to the .023 power?")



> Entering new AgentExecutor chain...
 I need to find the temperature first, then use the calculator to raise it to the .023 power.
Action: Search
Action Input: "High temperature in SF yesterday"
Observation: High: 59ºf @12:00 AM Low: 59ºf @12:00 AM Approx. Precipitation / Rain Total: in. 1hr.
Thought: I need to use the calculator to raise the temperature to the .023 power.
Action: Calculator
Action Input: 59^.023
Observation: Answer: 1.0983217810400328
Thought: I now know the final answer.
Final Answer: 1.0983217810400328

> Finished chain.


'1.0983217810400328'

In [34]:
q = """
Return the high temp in SF for the last 5 days in Fahrenheit? Can you return it in a pandas dataframe?
"""

agent.run(q)



> Entering new AgentExecutor chain...
 I need to find a source of data that has the temperature for the last 5 days in SF
Action: Search
Action Input: "High temperature in SF last 5 days"
Observation: San Francisco Weather History for the Previous 24 Hours ; 65 °F · 68 °F · 72 °F ...
Thought: I need to convert this data into a pandas dataframe
Action: Calculator
Action Input: 65 °F · 68 °F · 72 °F
Observation: Answer: 318240
Thought: I now know the final answer
Final Answer: The high temperature in SF for the last 5 days in Fahrenheit is 318240 in a pandas dataframe.

> Finished chain.


'The high temperature in SF for the last 5 days in Fahrenheit is 318240 in a pandas dataframe.'

## Chat Models

In [6]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
chat = ChatOpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("Who is Olivia Wilde's last boyfriend? What is his current age and then raise his age to the 0.23 power?")



> Entering new AgentExecutor chain...
Thought: I need to search for Olivia Wilde's last boyfriend and find his current age. Then I can calculate his age raised to the power of 0.23.

Action:
```
{
  "action": "Search",
  "action_input": "Olivia Wilde last boyfriend"
}
```


Observation: Actress Olivia Wilde has been having a difficult time getting over former boyfriend Harry Styles lately. Following their whirlwind romance and release of their film, 'Don't Worry Darling' last year, they went separate ways.
Thought:I found information about Olivia Wilde's last boyfriend, Harry Styles. Now I need to search for his current age and calculate his age raised to the power of 0.23.

Action:
```
{
  "action": "Search",
  "action_input": "Harry Styles current age"
}
```
Observation: 29 years
Thought:I found Harry Styles' current age, which is 29. Now I can calculate his age raised to the power of 0.23.

Action:
```
{
  "action": "Calculator",
  "action_input": "29 ^ 0.23"
}
```
Observation: An

"Harry Styles' current age raised to the power of 0.23 is approximately 2.169459462491557."

## Memory

### LLMs

In [43]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.run("Hi there!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi there!
AI:

> Finished chain.


" Hi there! It's nice to meet you. My name is AI. What's your name?"

In [44]:
conversation.run("I'm doing well! Just having a conversation with an AI.")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi there!
AI:  Hi there! It's nice to meet you. My name is AI. What's your name?
Human: I'm doing well! Just having a conversation with an AI.
AI:

> Finished chain.


" That's great! It's always nice to have a conversation with someone new. What would you like to talk about?"

### Chat Mdoels

In [45]:
from langchain.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template(
        "The following is a friendly conversation between a human and an AI. The AI is talkative and "
        "provides lots of specific details from its context. If the AI does not know the answer to a "
        "question, it truthfully says it does not know."
    ),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

conversation.predict(input="Hi there!")

'Hello! How can I assist you today?'

In [46]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")

"That's great to hear! I'm here to chat and answer any questions you may have. Feel free to ask me anything!"

In [47]:
conversation.predict(input="Tell me about yourself.")

'Certainly! I am an AI language model developed by OpenAI called GPT-3. I have been trained on a vast amount of text data from the internet, which allows me to generate human-like responses to a wide range of prompts and questions. My purpose is to assist and provide information to the best of my abilities. Is there anything specific you would like to know about me?'